In [3]:
import tensorflow as tf
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
print(tf. __version__)

2.11.0-dev20220812


In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

# whisper

In [5]:
import whisper

In [6]:
import gc
import torch
gc.collect()

torch.cuda.empty_cache()

In [46]:
torch.cuda.empty_cache()

In [2]:
model = whisper.load_model("small" , device="cuda")
result = model.transcribe("french.mp3" , task="transcribe")
text=result["text"]

In [3]:
print(text)

 Bonjour dans notre collège il y a au moins 600 élèves et 50 professeurs pour 40 salles et 10 salles spécifiques. Mon emploi du temps du lundi et je commence par français à 8h30 jusqu'à 9h20 puis anglais de 9h26 à 10h00 et ensuite de 10h35 à 11h30 français puis on mange 2h00. Après manger il y a sport soit gymnastique ou natation de 2h00 et on finit par maths qui commence à 15h50 et finit à 16h26. L'horaire d'une journée de jupes est de 8h30 à 16h26 Les matières et mes préférences sont ma matière préférée et le français parce que c'est intéressant. Celle que j'aime le moins c'est Histoire Géo je n'arrive pas à suivre et voilà. Les tenues des élèves sont ce qu'on veut. Il faut juste un peu respecter les règles que les tenues ne soient pas vulgaires. Le contenu de mon sac à dos d'une journée normale sur le lundi. Alors j'ai 4 cahiers, 1 cahier de français, 2 cahiers de maths et 1 cahier d'histoire. Je me trouve ma tablette, ma règle et voilà. Et mon carnet et mon agenda. Voilà. Au revoir

In [4]:
model = whisper.load_model("base")

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio(r"C:\Users\sridh\Desktop\audio\10sec.wav")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

Detected language: en
Life is a focus, life is a purpose. A purpose achieved by virtues which are developed with personal effort, do not gamble on life. It may not happen twice.


In [41]:
max(probs, key=probs.get)

'ja'

In [21]:
lang={'ja':'jpn_Jpan',
     'en':'eng_Latn'}

In [43]:
if max(probs, key=probs.get) in lang:
    print(lang[max(probs, key=probs.get)])

jpn_Jpan


# nllb

In [7]:
import fasttext

fasttext.FastText.eprint = lambda x: None

In [8]:
pretrained_lang_model = "lid218e.bin" # Path of model file

detect_model = fasttext.load_model(pretrained_lang_model)

#text="hello there"

#predictions = detect_model.predict(text, k=1)

#print(predictions) #(('__label__arb_Arab'), array([0.99960977]))

In [ ]:
input_lang = predictions[0][0].replace('__label__', '')

In [ ]:
input_lang

In [9]:
checkpoint = 'facebook/nllb-200-distilled-600M'

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

In [11]:
translate_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

C:\Users\sridh\anaconda3\envs\tf2.4_python3.9\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sridh\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
translator = pipeline('translation', model=translate_model, tokenizer=tokenizer, src_lang=input_lang, tgt_lang="jpn_Jpan", max_length = 400)

In [117]:
output = translator(text)
translated_text = output[0]['translation_text']
print(translated_text) 

こんにちは,私たちの大学には少なくとも600人の生徒と50人の教師が40部屋と10つの特定の部屋にいます.私の日程は月曜日です. 私は8時30分から9時20分までフランス語から始めます. 9時26分から10時30分まで英語です. それから10時35分から11時30分までフランス語です. それから2時食べます. 食べた後にスポーツがあります. 運動や2時泳ぎです. そして月曜日には数学で終わります. 15時50分から16時26分までです. 靴下の日程は8時30分から16時26分です. テーマと私の好みの内容は私の好みの内容です. そしてフランス語は,それが面白いからです. 私が一番好きなのは,歴史です. 私は到着し,それを守っています. さて,生徒たちは15時50分から16時00分まで,数学上の規則を尊重します. そして,私のスケジュール1日目です. 私は私のスケジュール1に,私のスケジュール1日目,そして私のスケジュール1に,私のスケジュール1の規則を遵守する必要があります.


# building gradio interface

In [21]:
import gradio as gr

In [18]:
model = whisper.load_model("base" , device="cuda")

In [25]:
def translate(text,out_lang):
    predictions = detect_model.predict(text, k=1)
    
    input_lang = predictions[0][0].replace('__label__', '')
    
    translator = pipeline('translation', model=translate_model, tokenizer=tokenizer, src_lang=input_lang, tgt_lang=out_lang, max_length = 400)
    
    output = translator(text)
    
    translated_text = output[0]['translation_text']
    
    return translated_text





def transcribe(audio,out_lang):
    #whisper
    audio = whisper.load_audio(audio)
    result = model.transcribe(audio , task="transcribe")
    translated=translate(result["text"],out_lang)
    return result["text"],translated

In [26]:
gr.Interface(
    title = 'translator (not so fast one😂)', 
    fn=transcribe, 
    inputs=[
        gr.inputs.Audio(source="microphone", type="filepath"),
        gr.Dropdown(["eng_Latn","jpn_Jpan", "spa_Latn", "fra_Latn"]),
    ],
    outputs=[
        gr.Textbox(label="text"),
        gr.Textbox(label="translated"),

    ]).launch()

C:\Users\sridh\AppData\Roaming\Python\Python39\site-packages\gradio\inputs.py:319: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
C:\Users\sridh\AppData\Roaming\Python\Python39\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


C:\Users\sridh\AppData\Roaming\Python\Python39\site-packages\gradio\processing_utils.py:241: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
